In [ ]:
import PIL
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw, ImageFont
import os

In [ ]:
tree = ET.parse("")
root = tree.getroot()

for obj in root.iter('object'):
    if obj.find('name').text == 'table':
        bndbox = obj.find('bndbox')
        table_bbox = (float(bndbox.find('xmin').text), float(bndbox.find('ymin').text),
                    float(bndbox.find('xmax').text), float(bndbox.find('ymax').text))
        break

img = Image.open("")

left, upper, right, lower = table_bbox
img_cropped = img.crop((left, upper, right, lower))
img_cropped.save('output1.jpg')

for obj in root.iter('object'):
    bndbox = obj.find('bndbox')
    x1 = float(bndbox.find('xmin').text) - left
    y1 = float(bndbox.find('ymin').text) - upper
    x2 = float(bndbox.find('xmax').text) - left
    y2 = float(bndbox.find('ymax').text) - upper

    bndbox.find('xmin').text = str(x1)
    bndbox.find('ymin').text = str(y1)
    bndbox.find('xmax').text = str(x2)
    bndbox.find('ymax').text = str(y2)

tree.write('output1.xml')

In [ ]:
def visualize_jpg(filename):
    img = PIL.Image.open(filename)
    img.show()

def visualize_xml(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    print("Thông tin trong file XML:")
    for child in root:
        print(child.tag, child.attrib)

if __name__ == "__main__":
    visualize_jpg("")
    visualize_xml("")

In [ ]:
def visualize_objects(image, objects):
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype("arial.ttf", 12)
    label_colors = {}

    for obj in objects:
        x1, y1, x2, y2 = obj.bounding_box
        label = obj.label

        if label in label_colors:
            color = label_colors[label]
        else:
            color = get_next_color(label_colors)
            label_colors[label] = color

        draw.rectangle([x1, y1, x2, y2], outline=color, width=2)

    for obj in objects:
        x, y = obj.center
        label = obj.label
        color = label_colors[label]
        draw.text((x, y), label, fill=color, font=font)

    return image

def get_next_color(label_colors):
    colors = [
        (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255), (255, 0, 255)
    ]
    return next(
        (color for color in colors if color not in label_colors.values()),
        (0, 0, 0),
    )
class Object:
    def __init__(self, element):
        self.label = element.find("name").text
        self.bounding_box = [
            float(element.find("bndbox").find("xmin").text),
            float(element.find("bndbox").find("ymin").text),
            float(element.find("bndbox").find("xmax").text),
            float(element.find("bndbox").find("ymax").text),
        ]
        self.center = (
            (self.bounding_box[0] + self.bounding_box[2]) / 2,
            (self.bounding_box[1] + self.bounding_box[3]) / 2,
        )

def main():
    image = PIL.Image.open("output1.jpg")

    tree = ET.parse("output1.xml")
    root = tree.getroot()

    objects = [Object(obj) for obj in root.findall("object")]
    image = visualize_objects(image, objects)

    image.show()

if __name__ == "__main__":
    main()

In [ ]:
img_dir = ''
xml_dir = ''
output_img_dir = ''
output_xml_dir = ''

os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_xml_dir, exist_ok=True)

for filename in os.listdir(img_dir):
    if filename.endswith('.jpg'):
        xml_filename = os.path.join(xml_dir, filename.replace('.jpg', '.xml'))
        tree = ET.parse(xml_filename)
        root = tree.getroot()

        for obj in root.iter('object'):
            if obj.find('name').text == 'table':
                bndbox = obj.find('bndbox')
                table_bbox = (float(bndbox.find('xmin').text), float(bndbox.find('ymin').text),
                            float(bndbox.find('xmax').text), float(bndbox.find('ymax').text))
                break

        img_filename = os.path.join(img_dir, filename)
        img = Image.open(img_filename)

        left, upper, right, lower = table_bbox
        img_cropped = img.crop((left, upper, right, lower))

        output_img_filename = os.path.join(output_img_dir, filename)
        img_cropped.save(output_img_filename)

        for obj in root.iter('object'):
            bndbox = obj.find('bndbox')
            x1 = float(bndbox.find('xmin').text) - left
            y1 = float(bndbox.find('ymin').text) - upper
            x2 = float(bndbox.find('xmax').text) - left
            y2 = float(bndbox.find('ymax').text) - upper

            bndbox.find('xmin').text = str(x1)
            bndbox.find('ymin').text = str(y1)
            bndbox.find('xmax').text = str(x2)
            bndbox.find('ymax').text = str(y2)

        output_xml_filename = os.path.join(output_xml_dir, filename.replace('.jpg', '.xml'))
        tree.write(output_xml_filename)

In [ ]:
img_dir = ""
xml_dir = ""
output_dir = ""

os.makedirs(output_dir, exist_ok=True)

colors = {'table': 'green', 'table row': 'red', 'table column': 'blue', 'table spanning cell': 'yellow'}

font = ImageFont.truetype('arial.ttf', 15)

for filename in os.listdir(img_dir):
    if filename.endswith('.jpg'):
        xml_filename = os.path.join(xml_dir, filename.replace('.jpg', '.xml'))
        tree = ET.parse(xml_filename)
        root = tree.getroot()

        img_filename = os.path.join(img_dir, filename)
        img = Image.open(img_filename)
        draw = ImageDraw.Draw(img)

        for obj in root.iter('object'):
            name = obj.find('name').text
            if name in colors:
                bndbox = obj.find('bndbox')
                bbox = [int(float(bndbox.find('xmin').text)), int(float(bndbox.find('ymin').text)),
                        int(float(bndbox.find('xmax').text)), int(float(bndbox.find('ymax').text))]
                draw.rectangle(bbox, outline=colors[name], width=2)
                draw.text((bbox[0], bbox[1] - 20), name, fill=colors[name], font=font)

        output_img_filename = os.path.join(output_dir, filename)
        img.save(output_img_filename)